In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess


class Spider12(scrapy.Spider):
    name = "Spider12"
    # Dominios a permitir
    allowed_domains = ["pagina12.com.ar"]
    # Configraciones
    custom_settings = {"FEED_FORMAT": "json", "FEED_URI": "resultados.json", "DEPTH_LIMIT": 2,
                       "FEED_EXPORT_ENCODING": "utf-8", }
    # URLs a crawlear
    start_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                  'https://www.pagina12.com.ar/secciones/economia',
                  'https://www.pagina12.com.ar/secciones/sociedad',
                  'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                  'https://www.pagina12.com.ar/secciones/el-mundo',
                  'https://www.pagina12.com.ar/secciones/deportes',
                  'https://www.pagina12.com.ar/secciones/cultura',
                  'https://www.pagina12.com.ar/secciones/contratapa']

    def parse(self, response):
        # Articulo promocionado
        nota_promocionada = response.xpath("//div[@class='featured-article__container']/h2/a/@href").get()
        if nota_promocionada is not None:
            # follow => se le indica que funcion debe continuar con la estraccion de la data
            yield response.follow(nota_promocionada, call_back=self.parse_nota)

        # Lista de noticias
        noticias = response.xpath("//ul[@class='article-list']//li//a/@href").getall()
        for noticia in noticias:
            yield response.follow(noticia, callback=self.parse_nota)

        # Link a la siguiente página
        next_page = response.xpath("//a[@class='pagination-btn-next']/@href")
        if next_page is not None:
            # Para este caso le indica que sea recursivo al llamar la misma funcion
            yield response.follow(next_page, callback=self.parse)

    def parse_nota(self, response):
        titulo = response.xpath("//div[@class='article-title']/text()").get()
        cuerpo = "".join(response.xpath("//div[@class='article-text']/p/text()").getall())
        # Una ves se extrae la data, se retorna el json, que se pusheara
        yield {"url": response.url, "titulo": titulo, "cuerpo": cuerpo}

# Inicia el proceso de crawling
process = CrawlerProcess()
# Se le pasa la clase con las configuraciones
process.crawl(Spider12)
# Inicia el proces
# Es mejor verlo por consola que con el notbbook
process.start